# Otimizando os hiper-parâmetros c/ Random Search

## Importando as libs

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

## 1. Aquisição dos dados

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

## 2. Pré-processamento

In [3]:
X_valid, X_train = X_train[:5000] / 255., X_train[5000:] / 255.
y_valid, y_train = y_train[:5000], y_train[5000:]
X_test = X_test / 255.

## 3. Construindo e treinando o modelo

In [4]:
def build_model(n_hidden=1, n_neurons=300, learning_rate=3e-3, input_shape=[28, 28]):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape= input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(10, activation="softmax"))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]) 
    return model

In [5]:
model = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [6]:
model.fit(X_train, y_train, epochs=30,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)]) 

Epoch 1/30
1719/1719 [==============================] - 3s 1ms/step - loss: 1.0136 - accuracy: 0.6967 - val_loss: 0.7062 - val_accuracy: 0.7788
Epoch 2/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6542 - accuracy: 0.7890 - val_loss: 0.5937 - val_accuracy: 0.8096
Epoch 3/30
1719/1719 [==============================] - 3s 1ms/step - loss: 0.5772 - accuracy: 0.8117 - val_loss: 0.5391 - val_accuracy: 0.8264
Epoch 4/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5363 - accuracy: 0.8222 - val_loss: 0.5117 - val_accuracy: 0.8350
Epoch 5/30
1719/1719 [==============================] - 3s 1ms/step - loss: 0.5105 - accuracy: 0.8270 - val_loss: 0.4934 - val_accuracy: 0.8372
Epoch 6/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4911 - accuracy: 0.8334 - val_loss: 0.4726 - val_accuracy: 0.8416
Epoch 7/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4764 - accuracy: 0.8373 - val_loss: 0.4693 - val_accuracy:

## 4. Otimizando os parâmetros com o RandomizedSearchCV

In [7]:
param_distribs = {
    "n_hidden": [0, 1, 2],
    "n_neurons": np.arange(268, 400).tolist(),
    "learning_rate": reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
}

rnd_search_cv = RandomizedSearchCV(model, param_distribs, n_iter=10, scoring='accuracy', cv=3, verbose=2)

In [8]:
rnd_search_cv.fit(X_train, y_train, epochs=30,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10), 
                             keras.callbacks.ModelCheckpoint("melhor_modelo_mlp_hiperparameters2.h5", save_best_only=True)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_neurons=348, n_hidden=2, learning_rate=0.001676333606797094 ...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/30
1146/1146 [==============================] - 3s 2ms/step - loss: 1.3477 - accuracy: 0.6283 - val_loss: 0.9106 - val_accuracy: 0.7060
Epoch 2/30
1146/1146 [==============================] - 3s 2ms/step - loss: 0.8121 - accuracy: 0.7340 - val_loss: 0.7252 - val_accuracy: 0.7618
Epoch 3/30
1146/1146 [==============================] - 3s 2ms/step - loss: 0.6916 - accuracy: 0.7737 - val_loss: 0.6455 - val_accuracy: 0.7886
Epoch 4/30
1146/1146 [==============================] - 3s 2ms/step - loss: 0.6285 - accuracy: 0.7935 - val_loss: 0.5965 - val_accuracy: 0.8058
Epoch 5/30
1146/1146 [==============================] - 3s 2ms/step - loss: 0.5879 - accuracy: 0.8059 - val_loss: 0.5640 - val_accuracy: 0.8128
Epoch 6/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.5582 - accuracy: 0.8144 - val_loss: 0.5543 - val_accuracy: 0.8186
Epoch 7/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.5359 - accuracy: 0.8211 - val_loss: 0.5200 - val_accuracy:

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.0min remaining:    0.0s


Epoch 1/30
1146/1146 [==============================] - 2s 2ms/step - loss: 1.3776 - accuracy: 0.6043 - val_loss: 0.9270 - val_accuracy: 0.7108
Epoch 2/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.8245 - accuracy: 0.7350 - val_loss: 0.7309 - val_accuracy: 0.7644
Epoch 3/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.6990 - accuracy: 0.7734 - val_loss: 0.6477 - val_accuracy: 0.7926
Epoch 4/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.6326 - accuracy: 0.7927 - val_loss: 0.5979 - val_accuracy: 0.8036
Epoch 5/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.5887 - accuracy: 0.8049 - val_loss: 0.5660 - val_accuracy: 0.8116
Epoch 6/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.5581 - accuracy: 0.8143 - val_loss: 0.5377 - val_accuracy: 0.8240
Epoch 7/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.5347 - accuracy: 0.8195 - val_loss: 0.5154 - val_accuracy:

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 24.3min finished


1719/1719 [==============================] - 4s 2ms/step - loss: 0.6552 - accuracy: 0.7806 - val_loss: 0.4885 - val_accuracy: 0.8348
Epoch 2/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4593 - accuracy: 0.8390 - val_loss: 0.4414 - val_accuracy: 0.8484
Epoch 3/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4159 - accuracy: 0.8529 - val_loss: 0.4111 - val_accuracy: 0.8610
Epoch 4/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3866 - accuracy: 0.8631 - val_loss: 0.3754 - val_accuracy: 0.8694
Epoch 5/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3659 - accuracy: 0.8702 - val_loss: 0.3646 - val_accuracy: 0.8734
Epoch 6/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3495 - accuracy: 0.8749 - val_loss: 0.3541 - val_accuracy: 0.8754
Epoch 7/30
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3356 - accuracy: 0.8807 - val_loss: 0.3331 - val_accuracy: 0.8802
Epo

RandomizedSearchCV(cv=3,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff3842e4d30>,
                   param_distributions={'learning_rate': [0.001864193848393307,
                                                          0.0008072900012393537,
                                                          0.0012243092482017548,
                                                          0.011437709044359923,
                                                          0.02752611895641111,
                                                          0.0010149500049642592,
                                                          0.00032885881372326175,
                                                          0.008400240707785623,
                                                          0.000651801158538200...
                                                          0.0005389914819176555,
                                                 

## 5. Avaliando o melhor modelo

In [9]:
best_model = keras.models.load_model("melhor_modelo_mlp_hiperparameters2.h5")

In [10]:
best_model.summary()

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_31 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_77 (Dense)             (None, 357)               280245    
_________________________________________________________________
dense_78 (Dense)             (None, 357)               127806    
_________________________________________________________________
dense_79 (Dense)             (None, 10)                3580      
Total params: 411,631
Trainable params: 411,631
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Meta: Acurácia:  0.8819 e Kappa:  0.8687777777777779
y_pred = np.argmax(best_model.predict(X_test), axis=-1)

In [23]:
from sklearn.metrics import accuracy_score, confusion_matrix, cohen_kappa_score
print('Acurácia: ', accuracy_score(y_test,y_pred))
print('Kappa: ', cohen_kappa_score(y_test,y_pred))
print('Matriz de confusão: \n', confusion_matrix(y_test,y_pred))

Acurácia:  0.8876
Kappa:  0.8751111111111112
Matriz de confusão: 
 [[820   0  13  27   6   1 126   0   7   0]
 [  3 967   0  22   3   0   5   0   0   0]
 [ 19   1 764  13 131   0  69   0   2   1]
 [ 13   3   9 900  33   0  38   0   4   0]
 [  1   1  59  29 858   0  50   0   2   0]
 [  0   0   0   1   0 966   0  19   1  13]
 [105   1  59  27  72   0 726   0  10   0]
 [  0   0   0   0   0  15   0 959   0  26]
 [  8   0   4   6   7   3   7   4 961   0]
 [  1   0   0   0   0   6   0  38   0 955]]


$Atividade (2.5):$ Ajuste os parâmetros de uma MLP profunda com o Random Search para o conjunto de dados MNIST e veja se você consegue obter melhorar a precisão. Em seguida, aplicar a estratégia do funil, onde caso a arquitetura testada possua mais de 1 camada oculta, que a próxima seja menor em número de neurônios que a anterior. Depois compare as estratégias das camadas ocultas com o mesmo número de neurônios com o funil.  